
https://platform.openai.com/api-keys

openai_info/api_info.json 에 API 키 입력

- 크레딧 필요

추론: FLASK/input_data 에 있는 flaks_evaluation_raw 수정

- (question_id) key 필요


평가: FLASK/evaluation_set 에 있는 flask_evaluation 수정

- (idx) key 필요

 최종적으로  FLASK/gpt_review/outputs/stats 에 분석결과가 CSV 파일로  저장됨
 - 최대 5점
1. content_skill.csv  - 8가지 영역평가
2. content_difficulty_skill.csv - 8가지 영역평가 , aggregate_difficulty_skill.py 수정 필요
3. content_domain.csv - 4가지 영역별 평가 , aggregate_domain.py 수정 필요


# 0.마운트/패키지설치

https://github.com/kaistAI/FLASK?tab=readme-ov-file

In [ ]:
!nvidia-smi

Mon Oct 28 18:08:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU ID를 0으로 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/FLASK/model_output")

In [ ]:
!pip install ray

In [ ]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
!huggingface-cli login #허깅페이스 토큰입력


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [ ]:
import os
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/FLASK/model_output'

In [ ]:
#!git clone https://huggingface.co/prometheus-eval/prometheus-7b-v2.0 #1번만 깃클론

# 1.모델추론 -응답생성

In [ ]:
"""
#원본 FLASK 1700개

os.chdir("/content/drive/MyDrive/Colab Notebooks/FLASK/model_output")

!python inference.py --model-path "/prometheus-7b-v2.0" --model-id "prometheus-7b-v2.0" --question-file "../input_data/flask_evaluation_raw.jsonl" --answer-file "/content/drive/MyDrive/test_answer1_raw.jsonl" --num-gpus 1

"""

'\n#원본 FLASK 1700개\n\nos.chdir("/content/drive/MyDrive/Colab Notebooks/FLASK/model_output")\n\n!python inference.py --model-path "/prometheus-7b-v2.0" --model-id "prometheus-7b-v2.0" --question-file "../input_data/flask_evaluation_raw.jsonl" --answer-file "/content/drive/MyDrive/test_answer1_raw.jsonl" --num-gpus 1\n\n'

In [ ]:
# 압축 FLASK 사용
os.chdir("/content/drive/MyDrive/Colab Notebooks/FLASK/model_output")

!python inference.py --model-path prometheus-7b-v2.0 --model-id prometheus-7b-v2.0 --question-file "../input_data/flask_evaluation_raw_10.jsonl" --answer-file "/content/drive/MyDrive/gpt4_answer.jsonl" --num-gpus 1

#question file : flask_evaluation_5QA.jsonl (압축본)
#answer file :  gpta4_answer.jsonl

#객체마다 question_id 키 있어야함


2024-10-28 17:37:14,236	INFO worker.py:1816 -- Started a local Ray instance.
  0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
(get_model_answers pid=39182) Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
(get_model_answers pid=39182) initial question The sentence you are given might be too wordy, complicated, or unclear. Rewrite the sentence and make your writing clearer by keeping it concise. Whenever possible, break complex sentences into multiple sentences and eliminate unnecessary words.
(get_model_answers pid=39182) Input: If you have any questions about my rate or if you find it necessary to increase or decrease the scope for this project, please let me know.
(get_model_answers pid=39182) The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a

# 2.모델 평가

In [ ]:
!pip install shortuuid

In [ ]:
!pip install openai

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/FLASK/gpt_review")

In [ ]:
#python gpt4_eval.py -q '../evaluation_set/flask_evaluation.jsonl' -a {answer_file} -o {output_review_file} -e {output_error_file}

!python gpt4_eval.py -q "../evaluation_set/flask_evaluation_10.jsonl" -a "/content/drive/MyDrive/gpt4_answer.jsonl" --output-review-file "/content/drive/MyDrive/gpt4_review.jsonl" -e "/content/drive/MyDrive/gpt4_error.jsonl"

#-a ,-o : 만들어진 a , o

# FLASK/evaluation_set/ ,  FLASK/input_data/  에 파일 넣어놔야함


We would like to request your feedback on the performance of the response of the assistant to the user instruction displayed below. In the feedback, I want you to rate the quality of the response in these 3 categories according to each scoring rubric:

Readability: Is the response structured to promote readability and coherence? Does the response exhibit excellent organization?
Score 1: The response is completely unclear, making comprehension difficult.
Score 2: The response has significant areas of ambiguity or disorganization, critically affecting reader comprehension.
Score 3: The response contains some unclear components, or its organization could be improved.
Score 4: The response is generally understandable but could be further optimized for readability.
Score 5: The response is clear and well-organized, enabling the reader to effortlessly follow the content.


Logical Correctness: Is the final answer provided by the response logically accurate and correct for an instruction that

# 3.분석

실행시 오류가 발생하므로

1. aggregate_difficulty_skill.py

2. aggregate_domain.py  파일의 수정이 필요함

결과적으로 FLASK/gpt_review/outputs/stats 에 분석결과가 CSV 파일로  저장됨

1. content_skill.csv
2. content_difficulty_skill.csv
3. content_domain.csv


In [ ]:
#cd gpt_review
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/FLASK/gpt_review")

In [ ]:
#python aggregate_skill.py -m {output_review_file}
!python aggregate_skill.py -m "/content/drive/MyDrive/gpt4_review.jsonl"

readability
logical correctness
conciseness
commonsense understanding
comprehension
factuality
insightfulness
completeness


In [ ]:
#python aggregate_difficulty_skill.py -m {output_review_file}
!python aggregate_difficulty_skill.py -m "/content/drive/MyDrive/gpt4_review.jsonl"

readability
logical correctness
conciseness
commonsense understanding
comprehension
factuality
insightfulness
completeness


In [ ]:
#python aggregate_domain.py -m {output_review_file}
!python aggregate_domain.py -m "/content/drive/MyDrive/gpt4_review.jsonl"

Humanities	128.0	36
Language	68.0	18
Culture	6.0	6
Technology	22.0	6


프로메테우스 답변예시:

1.The instruction to evaluate:

Explain the significance of Newton's laws of motion.

2.Response to evaluate:

Newton's laws describe how objects move and interact.

3.Reference Answer (Score 5):

Newton's laws explain the fundamental principles governing the motion of

objects in classical mechanics, with applications in various fields.

4.Score Rubrics: 1점~5점

Relevance to the topic, completeness, and accuracy.

* Score 1: Fails to address the question.

* Score 2: Partial or superficial response.

* Score 3: Adequate response but with minor omissions.

* Score 4: Thorough response with minor errors.

* Score 5: Comprehensive and highly accurate response.

5.Feedback:

The response provides a superficial understanding of Newton's laws without explaining the significance or impact. It merely states that the laws describe the movement and interaction of objects without offering any insight into their importance or application in various fields, thereby lacking depth and detail. This response does not completely address the instruction and fails to highlight the broader implications and relevance of the laws in different contexts, such as classical mechanics, engineering, physics, etc. So the overall score is 2.

[RESULT] 2</s>